## Notebook Objectives

Generate a Dataset to use in all future projetcs.

In [4]:
import numpy as np
import pandas as pd

In [5]:
# https://www.kaggle.com/datasets/bkcoban/customer-transactions/data
df_transactions = pd.read_csv('../data/transactions/sample_dataset.csv')
print(len(df_transactions))
df_transactions.head(3)

50000


,Customer ID,Name,Surname,Gender,Birthdate,Transaction Amount,Date,Merchant Name,Category
0,752858,Sean,Rodriguez,F,2002-10-20,35.47,2023-04-03,Smith-Russell,Cosmetic
1,26381,Michelle,Phelps,NaN,1985-10-24,2552.72,2023-07-17,"Peck, Spence and Young",Travel
2,305449,Jacob,Williams,M,1981-10-25,115.97,2023-09-20,Steele Inc,Clothing


### EDA

Finding Inconsistencies to adjust later.

In [6]:
print("1 Customer per Transaction. Let's make a Customer Table.")
df_transactions[['Customer ID', 'Name', 'Surname', 'Gender', 'Birthdate']].duplicated().sum()

1 Customer per Transaction. Let's make a Customer Table.


0

In [7]:
print("One Merchant can sell more then 1 category of product. Let's make a Merchant Table based only on ID.")
df_transactions[['Merchant Name', 'Category']].duplicated().sum()

One Merchant can sell more then 1 category of product. Let's make a Merchant Table based only on ID.


6121

### Creating Tables

tb_customer, tb_merchant, tb_transactions

In [8]:
# ----------------------------------------
# Create the customer dimension table
# ----------------------------------------
# Select customer-related attributes from the transaction table
# and remove duplicate records to ensure one row per customer
df_customer = df_transactions[
    ['Customer ID', 'Name', 'Surname', 'Gender', 'Birthdate']
].drop_duplicates()

# Persist the customer dimension table in Parquet format
df_customer.to_parquet("../data/transactions/tb_customer.parquet")


# ----------------------------------------
# Create the merchant dimension table
# ----------------------------------------
# Start from the merchant name column
df_merchant = df_transactions[['Merchant Name']]

# Create a surrogate key for merchants
# (using the row index as a simple unique identifier)
df_merchant['Merchant ID'] = df_transactions.index + 1

# Assign a country to each merchant based on a predefined distribution
df_merchant['Country'] = np.random.choice(
    ['Brazil', 'USA', 'China'],
    size=len(df_merchant),
    p=[0.7, 0.2, 0.1]
)

# Assign merchant size categories
df_merchant['Size'] = np.random.choice(
    ['Small', 'Medium', 'Large'],
    size=len(df_merchant),
    p=[0.3, 0.3, 0.4]
)

# Persist the merchant dimension table in Parquet format
df_merchant.to_parquet("../data/transactions/df_merchant.parquet")

C:\Users\preis\AppData\Local\Temp\ipykernel_32056\54378972.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_merchant['Merchant ID'] = df_transactions.index + 1
C:\Users\preis\AppData\Local\Temp\ipykernel_32056\54378972.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_merchant['Country'] = np.random.choice(
C:\Users\preis\AppData\Local\Temp\ipykernel_32056\54378972.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_i

In [22]:
# ----------------------------------------
# Steps to Create the transaction fact table
# ----------------------------------------

# ----------------------------------------
# 1. Define the number of transactions options
# ----------------------------------------
transaction_options = [
    1, 2, 3, 4, 5,
    6, 7, 8, 9, 10,
    11, 12, 13, 14, 15,
    16, 17, 18, 19, 25
]

# Business-driven probabilities
probabilities = (
    [0.20, 0.20] +                 # 1–2   → 40%
    [0.12, 0.12, 0.11] +           # 3–5   → 35%
    [0.04] * 5 +                   # 6–10  → 20%
    [0.005] * 10                   # 11–25 → 5%
)


# ----------------------------------------
# 2. Generate number of transactions per customer
# ----------------------------------------
df_customer['n_transactions'] = np.random.choice(
    transaction_options,
    size=len(df_customer),
    p=probabilities
)

# ----------------------------------------
# 3. Expand customers into transaction history
# ----------------------------------------
customer_ids = np.repeat(
    df_customer['Customer ID'].values,
    df_customer['n_transactions'].values
)

total_transactions = len(customer_ids)

# ----------------------------------------
# 4. Create the transaction fact table
# ----------------------------------------
tb_transactions = pd.DataFrame({
    'Transaction ID': range(1, total_transactions + 1),
    'Customer ID': customer_ids,
    'Merchant ID': np.random.choice(
        df_merchant['Merchant ID'],
        size=total_transactions
    ),
    'Transaction Value': np.round(
        np.random.lognormal(mean=3.5, sigma=0.8, size=total_transactions),
        2
    ),
    'Product Category': np.random.choice(
        df_transactions['Category'].unique(),
        size=total_transactions
    )
})

# ----------------------------------------
# 5. Generate random transaction dates
# ----------------------------------------
dates = pd.date_range(
    start='2025-01-01',
    end='2026-12-31',
    freq='D'
)

tb_transactions['Date'] = np.random.choice(
    dates,
    size=total_transactions
)

tb_transactions.to_parquet("../data/transactions/df_transactions.parquet")